## 前置准备

如果使用 docker image 启动，则不需要下面的准备步骤。否则按照下面的步骤准备环境。

<span style="color: rgb(50, 50, 50)">隐语使用 Ray 作为分布式计算调度框架，所以首先需要在 </span>alice 和 bob 节点分别启动 Ray 集群。

```
ray start --head --node-ip-address="ip" --port="port" --include-dashboard=False --disable-usage-stats
```

当屏幕输出中显示 <span style="color: rgb(0, 138, 0)">Ray runtime started</span>，则说明 Ray 的主节点启动成功。


## 初始化 SecretFlow

alice 和 bob 节点都需要初始化 secretflow。首先在两个节点分别选取一个可以被对方访问的地址，注意，端口号要选取未被占用的端口。


In [ ]:
import secretflow as sf
import spu

network_conf = {
    "parties": {
        "alice": {
            "address": "alice_address",
        },
        "bob": {
            "address": "bob_address",
        },
    },
}

### alice 初始化

1. 使用 alice Ray 主节点的 node-ip-address 和 port 填充 sf.init 的 address 参数。
2. self_party 字段填 alice
3. 注意下面初始化语句是只在 alice 节点执行。除了初始化代码，其他所有代码，会被同时下发到 alice 和 bob 节点执行。


In [ ]:
sf.shutdown()
sf.init(
    address="alice_ray_address",
    cluster_config={**network_conf, "self_party": "alice"},
    log_to_driver=True,
)

### bob 初始化

1. 使用 bob Ray 主节点的 node-ip-address 和 port 填充 sf.init 的 address 参数。
2. self_party 字段填 bob
3. 注意下面初始化语句是只在 bob 节点执行。除了初始化代码，其他所有代码，会被同时下发到 alice 和 bob 节点执行。


In [ ]:
sf.shutdown()
sf.init(
    address="bob_ray_address",
    cluster_config={**network_conf, "self_party": "bob"},
    log_to_driver=True,
)

## 初始化 SPU

1. alice 的 address 请填写可以被 bob 访通的地址，并且选择一个**未被占用的端口** ，注意不要和 Ray 端口冲突。
2. alice 的 listen_addr 可以和 alice address 里的端口一样。
3. bob 的 address 请填写可以被 alice 访通的地址，并且选择一个**未被占用的端口** ，注意不要和 Ray 端口冲突。
4. bob 的 listen_addr 可以和 bob address 里的端口一样。


In [ ]:
alice, bob = sf.PYU("alice"), sf.PYU("bob")
spu_conf = {
    "nodes": [
        {
            "party": "alice",
            "address": "alice_spu_address",
            "listen_addr": "alice_spu_address",
        },
        {
            "party": "bob",
            "address": "bob_spu_address",
            "listen_addr": "bob_spu_address",
        },
    ],
    "runtime_config": {
        "protocol": spu.spu_pb2.SEMI2K,
        "field": spu.spu_pb2.FM128,
        "sigmoid_mode": spu.spu_pb2.RuntimeConfig.SIGMOID_REAL,
    },
}
spu = sf.SPU(cluster_def=spu_conf)

## 隐私求交

<span style="color: rgb(50, 50, 50)">提供 </span>`psi_csv`<span style="color: rgb(50, 50, 50)"> 函数， </span>`psi_csv`<span style="color: rgb(50, 50, 50)"> 将 csv 文件作为输入，并在求交后生成 csv 文件。默认协议为 </span>[**KKRT**](https://eprint.iacr.org/2016/799.pdf)<span style="color: rgb(50, 50, 50)">。</span>


In [ ]:
import os

current_dir = os.getcwd()

input_path = {
    alice: f"{current_dir}/iris_alice.csv",
    bob: f"{current_dir}/iris_bob.csv",
}
output_path = {
    alice: f"{current_dir}/iris_alice_psi.csv",
    bob: f"{current_dir}/iris_alice_psi.csv",
}
spu.psi_csv("uid", input_path, output_path, "alice")